# PyTorchのトレーニング♨

## CNN - 初歩的なCNN 編

## [目次](TableOfContents.ipynb)
- [環境準備](#環境準備)
  - [インポート](#インポート)
  - [共通関数](#共通関数)
- [CNNアルゴリズム・モデル](#RNNアルゴリズム・モデル)
  - [MINSTデータ](#MINSTデータ)
  - [MINST-DNN](#MINST-DNN)
  - [MINST-CNN](#MINST-CNN)
  - [商品の自動タグ付け](#商品の自動タグ付け)

## 参考
開発基盤部会 Wiki
- データマイニング（DM）- Python - DL  
https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E3%83%87%E3%83%BC%E3%82%BF%E3%83%9E%E3%82%A4%E3%83%8B%E3%83%B3%E3%82%B0%EF%BC%88DM%EF%BC%89-%20Python%20-%20DL

## [環境準備](TensorFlowAndKeras0.ipynb)

### インポート

In [ ]:
import io
import requests

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn.metrics import confusion_matrix

import torch

# データの連結と分割
from torch.utils.data import ConcatDataset, Subset

#モデル定義ライブラリ
import torch.nn as nn
import torch.nn.functional as F
# 最適化関数ライブラリ
import torch.optim as optim
# ミニバッチ用ライブラリ
from torch.utils.data import DataLoader

import torchvision

# torchvision内のMNIST
from torchvision.datasets import MNIST
# ToTensorを含む前処理パッケージの呼び出し
import torchvision.transforms as transforms

# プログレスバー（学習関数で使用する
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

### 共通関数

#### 画像確認

##### 画像とラベルの確認

In [ ]:
def show_image_info(image_tuples, index):    
    image, label = image_tuples[index]
    print("label: ", label)
    print("Image: ")
    plt.imshow(image[0])
    plt.show()

##### ランダムな画像一覧

In [ ]:
def show_image_list(image_tuples, numOfCls=10, numOfImgInCls=10):
    pos = 1
    plt.figure(figsize=(numOfCls, numOfImgInCls))

    # クラス毎に以下の処理を繰り返す。
    for targetClass in range(numOfCls):
        targetIdx = []
        
        # 当該クラスの画像のインデックスリストを取得
        for i in range(len(image_tuples)):
            image, label = image_tuples[i]
            if label == targetClass:
                targetIdx.append(i)
        
        # 当該クラスのインデックスリストからランダムに選んだ最初のn個の画像を描画
        np.random.shuffle(targetIdx)
        for idx in targetIdx[:numOfImgInCls]:
            plt.subplot(numOfCls, numOfImgInCls, pos)
            image, label = image_tuples[idx]
            plt.imshow(image[0])
            plt.axis('off')
            pos += 1

plt.show()

##### 誤った推論の画像を表示する関数

In [ ]:
y = []
for i in range(len(org)):
    y.append(org[i][1])

print(len(y))
print(len(y_pred))
index = (np.array(y) == np.array(y_pred))
print(index)

In [ ]:
def show_incorrect_image_list(image_tuples, y_pred, numOfImg=10):
    #x = []
    y = []
    for i in range(len(image_tuples)):
        #x.append(image_tuples[i][0])
        y.append(image_tuples[i][1])
        
    index = (np.array(y) != np.array(y_pred))
    for i, val in enumerate(index):
        if val == True:
            print('i: ', i)
            print('predict: ', y_pred[i])
            print('answer : ', y[i])
            show_image_info(image_tuples, i)
            numOfImg -= 1
            if numOfImg <= 0:
                break

#### [分類問題関連](ScikitLearnTraining5.ipynb)

##### 分類問題のメトリック表示関数

In [ ]:
def print_metrics(label, pred):
    print('accuracy: %.3f' % metrics.accuracy_score(label, pred)) # 正答率
    
    print('\nmicro') # ミクロ平均
    print('recall: %.3f' % metrics.recall_score(label, pred, average='micro')) # 再現率
    print('precision: %.3f' % metrics.precision_score(label, pred, average='micro')) # 適合率
    print('f1_score: %.3f' % metrics.f1_score(label, pred, average='micro')) # f値
    
    print('\nmacro') # マクロ平均
    print('recall: %.3f' % metrics.recall_score(label, pred, average='macro')) # 再現率
    print('precision: %.3f' % metrics.precision_score(label, pred, average='macro')) # 適合率
    print('f1_score: %.3f' % metrics.f1_score(label, pred, average='macro')) # f値

##### 混同行列のグラフ化関数

In [ ]:
def plot_cm(confmat, label):
    numOfCls = len(label)
    fig, ax = plt.subplots(figsize=(numOfCls, numOfCls))
    ax.matshow(confmat, cmap=plt.cm.Blues, alpha=0.3)
    for i in range(confmat.shape[0]):
        for j in range(confmat.shape[1]):
            ax.text(x=j, y=i, s=confmat[i, j], va='center', ha='center')
            
    # 軸目盛を打つ場所を決める
    ax.set_xticks(np.arange(len(label)))
    ax.set_yticks(np.arange(len(label)))
    # 軸目盛を設定
    ax.set_xticklabels(label)
    ax.set_yticklabels(label)
    #plt.xticks(np.array(label)) # x軸の目盛りを指定
    #plt.yticks(np.array(label)) # y軸の目盛りを指定
    
    plt.xlabel('y_pred label')
    plt.ylabel('y label')
    plt.show()

#### 学習履歴表示関数

##### 損失

In [ ]:
def plot_history_loss(train_list, val_list):
    plt.plot(train_list, label=f"loss for training")
    plt.plot(val_list, label=f"loss for validation")
    
    plt.title(f"model loss")
    plt.xlabel("epoch")
    plt.ylabel(f"loss")
    plt.legend(loc='best')
    
    plt.show()

##### 正解率

In [ ]:
def plot_history_acc(train_list, val_list):
    plt.plot(train_list, label=f"acc for training")
    plt.plot(val_list, label=f"acc for validation")
    
    plt.title(f"model acc")
    plt.xlabel("epoch")
    plt.ylabel(f"acc")
    plt.legend(loc='best')
    
    plt.show()

## CNNアルゴリズム・モデル

### MINSTデータ

#### 生成

In [ ]:
# ToTensorを定義
transform = transforms.ToTensor()

# 初回はdownload=Trueで実行する。
# transformで、(高さ, 幅, チャネル数) → (チャネル数, 幅, 高さ)に変更
train_org = MNIST(root="./data", download=False, train=True, transform=transform)
test_org = MNIST(root="./data", download=False, train=False, transform=transform)
print(len(train_org))
print(len(test_org))

#### 確認
TensorFlow・Kerasと勝手が違い、PyTorchのMNISTデータは、  
(画像, 正解ラベル)の組み合わせのtupleの配列から構成される。 

In [ ]:
# 独特の型
print(type(train_org))
print(type(train_org[0]))

In [ ]:
# 先頭のデータを取り出す。
show_image_info(train_org, 0)

#### 加工
一応、結合データも作っておく（型変換、正規化は、ToTensorで処理済み）

In [ ]:
org = ConcatDataset([train_org, test_org])
print(type(org))
print(len(org))

#### 理解

In [ ]:
show_image_info(org, 10)

In [ ]:
show_image_list(test_org, 10, 10) # orgは重過ぎ

#### 準備

##### PyTorch入力用
型変換、正規化は、ToTensorで処理済みとのこと。

##### 正解ラベルのOne-Hotエンコーディング
Kerasで必要だった処理だがPyTorchでは不要らしい。

##### 学習・テストデータの分割

In [ ]:
train_data = Subset(train_org, np.arange(0, 50000))
val_data = Subset(train_org, np.arange(50000, 60000))
print(train_data, len(train_data))
print(val_data, len(val_data))

##### 結果が同じになるように乱数のシードを設定

In [ ]:
torch.manual_seed(1)

### MINST-DNN
- バージョンのバグか、何故か9を軒並み0と推論する。
- サンプルコードをフラットにコピペして検証しても結果変わらず。

#### モデリング

##### torch.nn.Moduleのサブクラスを定義してモデルを構築
一方通行ではない複雑なモデル（ネットワーク）の場合

In [ ]:
class Model(nn.Module):
    # 各層を__init__に定義する
    def __init__(self):
        # 親クラスの継承
        super(Model, self).__init__()
        
        # 全結合層の定義
        self.fc1 = nn.Linear(784, 900)
        self.fc2 = nn.Linear(900, 1000)
        self.fc3 = nn.Linear(1000, 500)
        self.output = nn.Linear(500, 10)
        
    # 順伝搬の計算の定義
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        
        return F.relu(self.output(x))

##### torch.nn.Sequentialでモデルを構築
一方通行のシンプルなモデル（ネットワーク）の場合

```python
class Model(nn.Module):
    # 各層を__init__に定義する
    def __init__(self):
        # 親クラスの継承
        super(Model, self).__init__()
        
        # nn.Sequentalで処理をまとめる
        self.layers = nn.Sequential(
            nn.Linear(784, 900),
            nn.ReLU(inplace=True),
            nn.Linear(900, 1000),
            nn.ReLU(inplace=True),
            nn.Linear(1000, 500),
            nn.ReLU(inplace=True),
            nn.Linear(500, 10),
            nn.ReLU(inplace=True)
        )
        
    # 順伝搬の計算の定義
    def forward(self, x):
        return self.layers(x)
```

##### ネットワーク生成

In [ ]:
model = Model()

##### 損失関数

In [ ]:
criterion = nn.CrossEntropyLoss()

##### 最適化関数

In [ ]:
# 最適化関数にネットワークのモデルのパラメータを渡す
# lr: 学習率，momentum: 慣性項 
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9) 

##### ミニバッチ

In [ ]:
# 学習データはバッチサイズ100でシャッフルする
train_loader = DataLoader(dataset=train_data, batch_size=100, shuffle=True) 
# 検証データはバッチサイズ100シャッフルしない
val_loader = DataLoader(dataset=val_data, batch_size=100, shuffle=False) 

##### train関数
1epochの処理

In [ ]:
def train(model, loader, criterion, optimizer):
    # 1epochの損失と精度の値を保存する変数を定義
    epoch_loss = 0.0
    epoch_acc = 0.0

    # 訓練モードに設定
    model.train()
    
    # データを取り出す
    for images, labels in tqdm(loader):
        # 画像のベクトル化
        images = images.view(-1, 28*28)
        
        # 画像を計算するためにCPUに送る
        images, labels = images.to("cpu"), labels.to("cpu")
        
        # 勾配をリセット(初期化)
        optimizer.zero_grad()
        
        # 順伝搬を計算(予測)
        outputs = model(images)
        # outputsから予測ラベルを計算
        # 最も予測率が高いものを予測ラベルとする
        _, preds = torch.max(outputs.data, 1)
        
        # 損失関数の計算
        loss = criterion(outputs, labels)
       
        #損失の和を求める
        epoch_loss += loss.item() * images.size(0)
        
        # 精度の和を求める
        epoch_acc += (preds == labels).sum().item()
        
        # 逆伝搬の計算
        loss.backward()
        
        # 重みの更新
        optimizer.step()
        
    # 損失と精度の平均値を返す
    data_size = len(loader.dataset)
    return epoch_loss/data_size, epoch_acc/data_size

##### validation関数
1epochの処理

In [ ]:
def validation(model, loader, criterion):
    # 1epochの損失と精度の値を保存する変数を定義
    epoch_loss = 0.0
    epoch_acc = 0.0

    # 評価モードに設定
    model.eval()
    
    # データを取り出す
    for images, labels in tqdm(loader):
        # 画像のベクトル化
        images = images.view(-1, 28*28)
        
        # 画像を計算するためにCPUに送る
        images, labels = images.to("cpu"), labels.to("cpu")
        
        # テスト時はno_gradを指定することによって、勾配計算を行わない
        with torch.no_grad():
            # 順伝搬を計算(予測)
            outputs = model(images)
            # outputsから予測ラベルを計算
            # 最も予測率が高いものを予測ラベルとする
            _, preds = torch.max(outputs.data, 1)
        
            # 損失関数の計算
            loss = criterion(outputs, labels)
       
        #損失の和を求める
        epoch_loss += loss.item() * images.size(0)
        
        # 精度の和を求める
        epoch_acc += (preds == labels).sum().item()
        
    # 損失と精度の平均値を返す
    data_size = len(loader.dataset)
    return epoch_loss/data_size, epoch_acc/data_size

#### 実行

##### 学習

```python
#epoch数
n_epoch = 20 
# 学習結果を保存するためのリスト
train_loss_list, train_acc_list = [], []
val_loss_list, val_acc_list = [], []

for epoch in range(n_epoch):
    # epochの表示
    print(f" {epoch+1}/{n_epoch}")
    
    #学習フェイズ
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    print(f"train_loss : {train_loss : .4f} train_acc : {train_acc : .4f}")
    # 計算結果を保存
    train_loss_list.append(train_loss), train_acc_list.append(train_acc)

    #検証フェイズ
    val_loss, val_acc = validation(model, val_loader, criterion)
    print(f"val_loss : {val_loss : .4f} val_acc : {val_acc : .4f}")
    # 計算結果を保存
    val_loss_list.append(val_loss), val_acc_list.append(val_acc)
```

###### モデルの保存

In [ ]:
#torch.save(model.state_dict(), '../work/minst-dnn.pth')

###### モデルの復元

In [ ]:
model = Model()
load_weights = torch.load('../work/minst-dnn.pth', map_location= "cpu")
model.load_state_dict(load_weights)

##### 推論

###### 個別

In [ ]:
index = 10
show_image_info(test_org, index)

image, answer = test_org[index]
# DataLoaderを通さない
image = image.unsqueeze(0)
# 画像のベクトル化
image = image.view(-1, 28*28)

# 画像の推論

# 評価モードに設定し
model.eval()

# パラメタを更新しないで順電波的な
with torch.no_grad():
    # 順伝搬を計算(予測)
    output = model(image)
    # outputsから予測ラベル（最も予測率が高い要素とインデックス）
    _, predict = torch.max(output.data, 1) # 1はaxis(行毎)

#予測結果をtoensorからintにキャスト
predict = int(predict)
print('predict: ', predict)
print('answer : ', answer)

if predict == answer:
    print('正解です｡')
else:
    print('不正解です')

###### 一括

In [ ]:
def predict(model, loader):
    # 正解ラベル, 推論ラベル
    y, y_pred = [], []
    
    # 推論
    model.eval()
    for _, (images, labels) in enumerate(loader):
        images = images.view(-1, 28*28)
        with torch.no_grad():
            outputs = model(images)
            # 正解ラベルと推論結果をリストに保存
            y += [int(l) for l in labels]
            y_pred += [int(l.argmax()) for l in outputs]

    return y, y_pred

#### 評価

##### 実測・予測を表示

In [ ]:
org_loader = DataLoader(dataset=org, batch_size=32, shuffle=False)
y, y_pred = predict(model, org_loader)
ret = (np.array(y) == np.array(y_pred))

##### [スコアを表示](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E3%83%87%E3%83%BC%E3%82%BF%E3%83%9E%E3%82%A4%E3%83%8B%E3%83%B3%E3%82%B0%EF%BC%88DM%EF%BC%89-%20CRISP-DM#uf759972)

In [ ]:
(len(np.where(ret==True)[0]) / ret.size)

In [ ]:
train_loader = DataLoader(dataset=train_org, batch_size=32, shuffle=False)
y, y_pred = predict(model, train_loader)
ret = (np.array(y) == np.array(y_pred))
(len(np.where(ret==True)[0]) / ret.size)

In [ ]:
test_loader = DataLoader(dataset=test_org, batch_size=32, shuffle=False)
y, y_pred = predict(model, test_loader)
ret = (np.array(y) == np.array(y_pred))
(len(np.where(ret==True)[0]) / ret.size)

##### 混同行列
間違い易い組合せが解る。

In [ ]:
cm_minst = confusion_matrix(y, y_pred)
plot_cm(cm_minst, [0,1,2,3,4,5,6,7,8,9])

###### 混同行列のメトリクス

In [ ]:
print_metrics(y, y_pred)

###### 間違った組み合わせの表示

In [ ]:
show_incorrect_image_list(test_org, y_pred, 5)

##### 学習履歴を表示

In [ ]:
# 損失値(Loss)の遷移のプロット
#plot_history_loss(train_loss_list, val_loss_list)
# 精度(Acc)の遷移のプロット
#plot_history_acc(train_acc_list, val_acc_list)

### MINST-CNN

#### モデリング

##### torch.nn.Moduleのサブクラスを定義してモデルを構築
一方通行ではない複雑なモデル（ネットワーク）の場合

##### torch.nn.Sequentialでモデルを構築
一方通行のシンプルなモデル（ネットワーク）の場合

In [ ]:
class CNN(nn.Module):

    def __init__(self):
        super(CNN, self).__init__() # nn.Moduleを継承する

        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=1), # 出力サイズ: チャネル=16, 高さ=27, 幅=27
            nn.BatchNorm2d(16)
        )
        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=1), # 出力サイズ: チャネル=32, 高さ=26, 幅=26
            nn.BatchNorm2d(32)
        )
        self.full_connection = nn.Sequential(
            nn.Linear(in_features=32*26*26, out_features=512), # in_featuresは直前の出力ユニット数
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(in_features=512, out_features=10)
        )


    # Forward計算の定義
    # 参考：Define by Runの特徴（入力に合わせてForward計算を変更可）
    def forward(self, x):

        x = self.block1(x)
        x = self.block2(x)

        # 直前のMaxPoolの出力が2次元（×チャネル数）
        # なので，全結合の入力形式に変換
        # 参考：KerasのFlatten()と同じような処理
        x = x.view(x.size(0), 32 * 26 * 26)

        y = self.full_connection(x)
        
        return y

##### ネットワーク生成

In [ ]:
model = CNN()

##### 損失関数

In [ ]:
criterion = nn.CrossEntropyLoss()

##### 最適化関数

In [ ]:
# 最適化関数にネットワークのモデルのパラメータを渡す
# lr: 学習率，momentum: 慣性項 
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9) 

##### ミニバッチ

In [ ]:
# 学習データはバッチサイズ100でシャッフルする
train_loader = DataLoader(dataset=train_data, batch_size=100, shuffle=True) 
# 検証データはバッチサイズ100シャッフルしない
val_loader = DataLoader(dataset=val_data, batch_size=100, shuffle=False) 

##### train関数
1epochの処理

In [ ]:
def train(model, loader, criterion, optimizer):
    # 1epochの損失と精度の値を保存する変数を定義
    epoch_loss = 0.0
    epoch_acc = 0.0

    # 訓練モードに設定
    model.train()
    
    # データを取り出す
    for images, labels in tqdm(loader):
        # 画像を計算するためにCPUに送る
        images, labels = images.to("cpu"), labels.to("cpu")
        
        # 勾配をリセット(初期化)
        optimizer.zero_grad()
        
        # 順伝搬を計算(予測)
        outputs = model(images)
        # outputsから予測ラベルを計算
        # 最も予測率が高いものを予測ラベルとする
        _, preds = torch.max(outputs.data, 1)
        
        # 損失関数の計算
        loss = criterion(outputs, labels)
       
        #損失の和を求める
        epoch_loss += loss.item() * images.size(0)
        
        # 精度の和を求める
        epoch_acc += (preds == labels).sum().item()
        
        # 逆伝搬の計算
        loss.backward()
        
        # 重みの更新
        optimizer.step()
        
    # 損失と精度の平均値を返す
    data_size = len(loader.dataset)
    return epoch_loss/data_size, epoch_acc/data_size

##### validation関数
1epochの処理

In [ ]:
def validation(model, loader, criterion):
    # 1epochの損失と精度の値を保存する変数を定義
    epoch_loss = 0.0
    epoch_acc = 0.0

    # 評価モードに設定
    model.eval()
    
    # データを取り出す
    for images, labels in tqdm(loader):
        # 画像を計算するためにCPUに送る
        images, labels = images.to("cpu"), labels.to("cpu")
        
        # テスト時はno_gradを指定することによって、勾配計算を行わない
        with torch.no_grad():
            # 順伝搬を計算(予測)
            outputs = model(images)
            # outputsから予測ラベルを計算
            # 最も予測率が高いものを予測ラベルとする
            _, preds = torch.max(outputs.data, 1)
        
            # 損失関数の計算
            loss = criterion(outputs, labels)
       
        #損失の和を求める
        epoch_loss += loss.item() * images.size(0)
        
        # 精度の和を求める
        epoch_acc += (preds == labels).sum().item()
        
    # 損失と精度の平均値を返す
    data_size = len(loader.dataset)
    return epoch_loss/data_size, epoch_acc/data_size

#### 実行

##### 学習

```python
#epoch数
n_epoch = 20 
# 学習結果を保存するためのリスト
train_loss_list, train_acc_list = [], []
val_loss_list, val_acc_list = [], []

for epoch in range(n_epoch):
    # epochの表示
    print(f" {epoch+1}/{n_epoch}")
    
    #学習フェイズ
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    print(f"train_loss : {train_loss : .4f} train_acc : {train_acc : .4f}")
    # 計算結果を保存
    train_loss_list.append(train_loss), train_acc_list.append(train_acc)

    #検証フェイズ
    val_loss, val_acc = validation(model, val_loader, criterion)
    print(f"val_loss : {val_loss : .4f} val_acc : {val_acc : .4f}")
    # 計算結果を保存
    val_loss_list.append(val_loss), val_acc_list.append(val_acc)
```

###### モデルの保存
結構時間がかかるのでモデルを保存しておく。

In [ ]:
#torch.save(model.state_dict(), '../work/minst-cnn.pth')

###### モデルの復元

In [ ]:
model = CNN()
load_weights = torch.load('../work/minst-cnn.pth', map_location= "cpu")
model.load_state_dict(load_weights)

##### 推論

###### 個別

In [ ]:
index = 10
show_image_info(test_org, index)

image, answer = test_org[index]
# DataLoaderを通さない
image = image.unsqueeze(0)

# 画像の推論

# 評価モードに設定し
model.eval()

# パラメタを更新しないで順電波的な
with torch.no_grad():
    # 順伝搬を計算(予測)
    output = model(image)
    # outputsから予測ラベル（最も予測率が高い要素とインデックス）
    _, predict = torch.max(output.data, 1) # 1はaxis(行毎)

#予測結果をtoensorからintにキャスト
predict = int(predict)
print('predict: ', predict)
print('answer : ', answer)

if predict == answer:
    print('正解です｡')
else:
    print('不正解です')

###### 一括

In [ ]:
def predict(model, loader):
    # 正解ラベル, 推論ラベル
    y, y_pred = [], []
    
    # 推論
    model.eval()
    for _, (images, labels) in enumerate(loader):
        with torch.no_grad():
            outputs = model(images)
            # 正解ラベルと推論結果をリストに保存
            y += [int(l) for l in labels]
            y_pred += [int(l.argmax()) for l in outputs]

    return y, y_pred

#### 評価

##### 実測・予測を表示

In [ ]:
org_loader = DataLoader(dataset=org, batch_size=32, shuffle=False)
y, y_pred = predict(model, org_loader)
ret = (np.array(y) == np.array(y_pred))

##### [スコアを表示](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E3%83%87%E3%83%BC%E3%82%BF%E3%83%9E%E3%82%A4%E3%83%8B%E3%83%B3%E3%82%B0%EF%BC%88DM%EF%BC%89-%20CRISP-DM#uf759972)

In [ ]:
(len(np.where(ret==True)[0]) / ret.size)

In [ ]:
train_loader = DataLoader(dataset=train_org, batch_size=32, shuffle=False)
y, y_pred = predict(model, train_loader)
ret = (np.array(y) == np.array(y_pred))
(len(np.where(ret==True)[0]) / ret.size)

In [ ]:
test_loader = DataLoader(dataset=test_org, batch_size=32, shuffle=False)
y, y_pred = predict(model, test_loader)
ret = (np.array(y) == np.array(y_pred))
(len(np.where(ret==True)[0]) / ret.size)

##### 混同行列
間違い易い組合せが解る。

In [ ]:
cm_minst = confusion_matrix(y, y_pred)
plot_cm(cm_minst, [0,1,2,3,4,5,6,7,8,9])

###### 混同行列のメトリクス

In [ ]:
print_metrics(y, y_pred)

###### 間違った組み合わせの表示

In [ ]:
show_incorrect_image_list(test_org, y_pred, 5)

##### 学習履歴を表示

In [ ]:
# 損失値(Loss)の遷移のプロット
#plot_history_loss(train_loss_list, val_loss_list)
# 精度(Acc)の遷移のプロット
#plot_history_acc(train_acc_list, val_acc_list)